In [1]:
# from .db_util import execute_query,write_to_article
# from ..settings import settings
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
# import psycopg2
# from psycopg2 import Error
# 结合 Selenium 和 Trafilatura 的完整解决方案
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
# from openai import OpenAI

def selenium_trafilatura_extract(url, driver=None, wait_time=3, save_html=False):
    """
    使用 Selenium 获取网页 HTML，然后用 Trafilatura 提取中文内容
    
    Args:
        url (str): 目标网页URL
        driver: Selenium WebDriver对象，如果为None会创建新的
        wait_time (int): 页面加载等待时间（秒）
        save_html (bool): 是否保存HTML文件到本地
    
    Returns:
        dict: 包含提取内容和元数据的字典
    """
    
    # 检查是否安装了trafilatura
    try:
        import trafilatura
    except ImportError:
        return {
            'success': False,
            'error': 'trafilatura未安装，请运行: pip install trafilatura',
            'url': url
        }
    
    # 创建或使用现有的driver
    driver_created = False
    if driver is None:
        try:
            from selenium import webdriver
            options = webdriver.ChromeOptions()
            options.add_argument('--headless')  # 无头模式，提高效率
            options.add_argument('--no-sandbox')
            options.add_argument('--disable-dev-shm-usage')
            options.add_argument('--disable-gpu')
            options.add_argument('--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36')
            
            driver = webdriver.Chrome(options=options)
            driver_created = True
            print(f"✅ 创建了新的Chrome Driver")
        except Exception as e:
            return {
                'success': False,
                'error': f'无法创建WebDriver: {str(e)}',
                'url': url
            }
    
    try:
        print(f"🌐 正在访问: {url}")
        
        # 访问网页
        driver.get(url)
        
        # 等待页面加载
        time.sleep(wait_time)
        
        # 等待body元素加载完成
        try:
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.TAG_NAME, "body"))
            )
        except:
            print("⚠️ 页面加载可能不完整，继续处理...")
        
        # 获取页面标题
        page_title = driver.title
        print(f"📄 页面标题: {page_title}")
        
        # 获取完整的HTML源码
        html_content = driver.page_source
        print(f"📝 HTML长度: {len(html_content)} 字符")
        
        # 保存HTML文件（可选）
        html_file_path = None
        if save_html:
            from urllib.parse import urlparse
            import re
            
            # 从URL生成文件名
            parsed_url = urlparse(url)
            filename = re.sub(r'[^a-zA-Z0-9\u4e00-\u9fff]', '_', parsed_url.netloc + parsed_url.path)
            html_file_path = f"{filename[:50]}.html"
            
            try:
                with open(html_file_path, 'w', encoding='utf-8') as f:
                    f.write(html_content)
                print(f"💾 HTML已保存到: {html_file_path}")
            except Exception as e:
                print(f"⚠️ HTML保存失败: {e}")
        
        # 使用 Trafilatura 提取内容
        print("🔍 使用 Trafilatura 提取内容...")
        
        # 基础文本提取
        extracted_text = trafilatura.extract(html_content)
        
        # 提取元数据
        metadata = trafilatura.extract_metadata(html_content)
        
        # 高级提取（包含更多选项）
        advanced_text = trafilatura.extract(
            html_content,
            favor_precision=True,    # 提高精确度
            favor_recall=False,      # 降低召回率，提高质量
            include_comments=False,  # 不包含评论
            include_tables=True,     # 包含表格
            include_formatting=True, # 保留基本格式
            include_links=False,     # 不包含链接
            include_images=False     # 不包含图片
        )
        
        # 提取结构化数据
        structured_data = trafilatura.extract(
            html_content,
            output_format='json',    # JSON格式输出
            include_formatting=True
        )
        
        # 组织返回结果
        result = {
            'success': True,
            'url': url,
            'page_title': page_title,
            'selenium_title': page_title,
            'extracted_title': metadata.title if metadata else page_title,
            'basic_text': extracted_text,
            'advanced_text': advanced_text,
            'structured_data': structured_data,
            'metadata': {
                'title': metadata.title if metadata else None,
                'author': metadata.author if metadata else None,
                'date': metadata.date if metadata else None,
                'description': metadata.description if metadata else None,
                'sitename': metadata.sitename if metadata else None,
                'language': metadata.language if metadata else None,
                'url': metadata.url if metadata else url
            },
            'statistics': {
                'html_length': len(html_content),
                'basic_text_length': len(extracted_text) if extracted_text else 0,
                'advanced_text_length': len(advanced_text) if advanced_text else 0,
                'has_content': bool(extracted_text and len(extracted_text) > 50)
            },
            'html_file_path': html_file_path,
            'processing_time': time.time()
        }
        
        # 输出提取结果摘要
        if extracted_text:
            print(f"✅ 内容提取成功!")
            print(f"   基础提取: {len(extracted_text)} 字符")
            print(f"   高级提取: {len(advanced_text) if advanced_text else 0} 字符")
            print(f"   提取标题: {result['extracted_title']}")
            print(f"   内容预览: {extracted_text[:100]}...")
        else:
            print(f"❌ 内容提取失败或内容为空")
            result['success'] = False
            result['error'] = '提取的内容为空'
        
        return result
        
    except Exception as e:
        error_msg = f"处理过程中出错: {str(e)}"
        print(f"❌ {error_msg}")
        return {
            'success': False,
            'error': error_msg,
            'url': url
        }
    
    finally:
        # 如果是创建的新driver，则关闭它
        if driver_created and driver:
            try:
                driver.quit()
                print("🚪 已关闭WebDriver")
            except:
                pass

# 批量处理函数
def batch_selenium_trafilatura_extract(urls, driver=None, wait_time=3, max_urls=5):
    """
    批量处理多个URL
    
    Args:
        urls (list): URL列表
        driver: 共享的WebDriver对象
        wait_time (int): 每个页面的等待时间
        max_urls (int): 最大处理URL数量
    
    Returns:
        list: 处理结果列表
    """
    
    results = []
    urls_to_process = urls[:max_urls]
    
    print(f"🚀 开始批量处理 {len(urls_to_process)} 个URL")
    print("=" * 60)
    
    # 如果没有提供driver，创建一个共享的
    driver_created = False
    if driver is None:
        try:
            from selenium import webdriver
            options = webdriver.ChromeOptions()
            options.add_argument('--headless')
            options.add_argument('--no-sandbox')
            options.add_argument('--disable-dev-shm-usage')
            
            driver = webdriver.Chrome(options=options)
            driver_created = True
            print("✅ 创建共享WebDriver")
        except Exception as e:
            print(f"❌ 无法创建WebDriver: {e}")
            return []
    
    try:
        for i, url in enumerate(urls_to_process, 1):
            print(f"\\n📍 处理第 {i}/{len(urls_to_process)} 个URL")
            print("-" * 40)
            
            result = selenium_trafilatura_extract(
                url, 
                driver=driver, 
                wait_time=wait_time, 
                save_html=False
            )
            
            results.append(result)
            
            # 避免过快请求
            if i < len(urls_to_process):
                time.sleep(1)
    
    finally:
        if driver_created and driver:
            driver.quit()
            print("\\n🚪 已关闭共享WebDriver")
    
    # 统计结果
    successful = sum(1 for r in results if r.get('success', False))
    print(f"\\n📊 批量处理完成: {successful}/{len(urls_to_process)} 成功")
    
    return results





        


In [5]:
# 检查Chrome和ChromeDriver版本兼容性的测试代码
import subprocess
import sys
import time

def check_chrome_version():
    """检查Chrome浏览器版本"""
    try:
        # Windows下检查Chrome版本的命令
        result = subprocess.run([
            'reg', 'query', 
            'HKEY_CURRENT_USER\\Software\\Google\\Chrome\\BLBeacon', 
            '/v', 'version'
        ], capture_output=True, text=True, shell=True)
        
        if result.returncode == 0:
            version_line = result.stdout.strip().split('\n')[-1]
            version = version_line.split()[-1]
            print(f"🔍 Chrome浏览器版本: {version}")
            return version
        else:
            print("❌ 无法检测Chrome版本")
            return None
    except Exception as e:
        print(f"❌ 检查Chrome版本时出错: {e}")
        return None

def check_chromedriver_version():
    """检查ChromeDriver版本"""
    try:
        result = subprocess.run(['chromedriver', '--version'], 
                              capture_output=True, text=True)
        if result.returncode == 0:
            version_info = result.stdout.strip()
            print(f"🔍 ChromeDriver版本: {version_info}")
            return version_info
        else:
            print("❌ ChromeDriver未找到或无法执行")
            return None
    except Exception as e:
        print(f"❌ 检查ChromeDriver版本时出错: {e}")
        return None

def quick_selenium_test():
    """快速测试Selenium启动时间"""
    print("🧪 开始快速启动测试...")
    
    options = webdriver.ChromeOptions()
    # 最小化配置，只保留必要选项
    options.add_argument('--headless')  # 无头模式，最快
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('--disable-gpu')
    options.add_argument('--disable-extensions')
    options.add_argument('--disable-logging')
    
    start_time = time.time()
    try:
        test_driver = webdriver.Chrome(options=options)
        end_time = time.time()
        print(f"✅ 无头模式启动成功，耗时: {end_time - start_time:.2f} 秒")
        
        # 测试简单页面访问
        test_start = time.time()
        test_driver.get("about:blank")
        test_end = time.time()
        print(f"✅ 页面访问测试成功，耗时: {test_end - test_start:.2f} 秒")
        
        test_driver.quit()
        return True
    except Exception as e:
        print(f"❌ 启动测试失败: {e}")
        return False

# 运行检查
print("=" * 50)
print("🔧 Chrome和ChromeDriver兼容性检查")
print("=" * 50)

check_chrome_version()
check_chromedriver_version()
quick_selenium_test()

print("\n💡 如果版本不兼容，请:")
print("1. 更新Chrome浏览器到最新版本")
print("2. 下载对应版本的ChromeDriver:")
print("   https://chromedriver.chromium.org/downloads")
print("3. 或使用webdriver-manager自动管理:")
print("   pip install webdriver-manager")


🔧 Chrome和ChromeDriver兼容性检查
🔍 Chrome浏览器版本: 137.0.7151.120
❌ 检查ChromeDriver版本时出错: [WinError 2] 系统找不到指定的文件。
🧪 开始快速启动测试...
✅ 无头模式启动成功，耗时: 1.52 秒
✅ 页面访问测试成功，耗时: 0.01 秒

💡 如果版本不兼容，请:
1. 更新Chrome浏览器到最新版本
2. 下载对应版本的ChromeDriver:
   https://chromedriver.chromium.org/downloads
3. 或使用webdriver-manager自动管理:
   pip install webdriver-manager


In [ ]:
# 优化Chrome启动配置，提高启动速度
options = webdriver.ChromeOptions()
# 基础性能优化选项
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--disable-gpu')
options.add_argument('--disable-web-security')
options.add_argument('--disable-features=VizDisplayCompositor')
# 网络和更新相关优化
options.add_argument('--disable-background-networking')
options.add_argument('--disable-background-timer-throttling')
options.add_argument('--disable-backgrounding-occluded-windows')
options.add_argument('--disable-component-update')
options.add_argument('--disable-default-apps')
options.add_argument('--disable-extensions')
# 日志和崩溃报告优化
options.add_argument('--disable-logging')
options.add_argument('--disable-crash-reporter')
# 如果不需要界面可以启用无头模式（会更快）
# options.add_argument('--headless')

with open('selenium_log.txt', 'a', encoding='utf-8') as f:
    f.write("🚀 正在启动Chrome浏览器...\n")
start_time = time.time()
driver = webdriver.Chrome(options=options)
end_time = time.time()
with open('selenium_log.txt', 'a', encoding='utf-8') as f:
    f.write(f"✅ Chrome启动完成，耗时: {end_time - start_time:.2f} 秒\n")

driver.implicitly_wait(0.5)
#进入首页找到热点页面
#进入百度页面
driver.get("https://www.baidu.com")
# 使用CSS选择器定位a标签，然后获取href属性
# 根据图片中的HTML结构，a标签有class="title-content tag-width c-link c-font-medium c-line-clamp1"
#urls = driver.find_elements(by=By.CSS_SELECTOR, value="a.title-content.tag-width.c-link.c-font-medium.c-line-clamp1")
urls = driver.find_elements(by=By.CSS_SELECTOR, value="a.title-content.c-link.c-font-medium.c-line-clamp1")
#urls =urls+urls2
specific_news = {}
#获取百度首页的热点url列表
hot_href_list = []  #每一个打开都是具体热点新闻页面
hot_text_list = []
specific_news = {}
for url in urls:
    text = url.text
    href = url.get_attribute('href')
    hot_href_list.append(href)
    hot_text_list.append(text)
href_value_list=[]
for url,hot_text in zip(hot_href_list,hot_text_list):
    #打开热点列表中的一个
    driver.get(url)
    time.sleep(1)
    #获取"更多消息"的url
    # 获取页面第一个h3标签（class='t'），然后获取该h3标签中的a标签的href值
    #这是普通的查看更多消息url
    try:
            # 使用CSS选择器定位第一个class为't'的h3标签
        h3_element = driver.find_element(by=By.CSS_SELECTOR, value="h3.t")
        
        # 在h3标签内查找a标签
        a_element = h3_element.find_element(by=By.TAG_NAME, value="a")
        
        # 获取a标签的href属性值
        href_value = a_element.get_attribute('href')
        # href_value_list.append(href_value)
        with open('selenium_log.txt', 'a', encoding='utf-8') as f:
            f.write(f'更多消息 {href_value}\n')
    except Exception as e:
        with open('selenium_log.txt', 'a', encoding='utf-8') as f:
            f.write(f"获取元素时出错: {e}\n")
        href_value = None
    if href_value:
        driver.get(href_value)
        with open('selenium_log.txt', 'a', encoding='utf-8') as f:
            f.write(f'进入热点{hot_text}页面\n')
        title = driver.title
        with open('selenium_log.txt', 'a', encoding='utf-8') as f:
            f.write(f"更多消息页面标题: {title}\n")
        # 获取页面所有包含aria-label属性的元素
        elements_with_aria_label = driver.find_elements(by=By.CSS_SELECTOR, value="[aria-label]")

        with open('selenium_log.txt', 'a', encoding='utf-8') as f:
            f.write(f"找到 {len(elements_with_aria_label)} 个包含aria-label属性的元素\n")
            f.write("=" * 80 + "\n")
        single_new_list=[]
        # 遍历每个元素并打印其原始HTML代码
        for i, element in enumerate(elements_with_aria_label, 1):
            # try:
            #     # 获取元素的aria-label属性值
            #     aria_label_value = element.get_attribute('aria-label')
            #     # 获取元素的原始HTML代码
            #     html_code = element.get_attribute('outerHTML')
                
            #     # 将数据写入txt文件
            #     with open('aria_label_data.txt', 'a', encoding='utf-8') as f:
            #         f.write(f"元素 {i}:\n")
            #         f.write(f"aria-label值: {aria_label_value}\n")
            #         f.write(f"HTML代码: {html_code}\n")
            #         f.write("-" * 50 + "\n")
            # except Exception as e:
            #     print(f"处理第 {i} 个元素时出错: {e}")
            #     print("-" * 50)    
            # 检查元素结构，寻找a标签-span标签-span标签-span标签的模式
            if i >= 2:  # 从第三个元素开始检查
                # 检查当前元素及其后3个元素是否形成a-span-span-span模式
                if i + 3 <= len(elements_with_aria_label):  # 确保有足够的后续元素
                    current_element = element
                    next_element_1 = elements_with_aria_label[i]  # 当前元素
                    next_element_2 = elements_with_aria_label[i+1]  # 后一个元素
                    next_element_3 = elements_with_aria_label[i+2]  # 后两个元素
                    
                    # 检查标签类型
                    current_tag = current_element.tag_name
                    next_tag_1 = next_element_1.tag_name
                    next_tag_2 = next_element_2.tag_name
                    next_tag_3 = next_element_3.tag_name
                    
                    # 检查是否符合 a-span-span-span 的模式
                    if (current_tag == 'a' and next_tag_1 == 'span' and 
                        next_tag_2 == 'span' and next_tag_3 == 'span'):
                        
                        try:
                            # 获取第一个a标签的href和text
                            a_href = current_element.get_attribute('href')
                            a_text = current_element.text.strip()
                            
                            # 获取第二个span标签的aria-label值
                            second_span_aria_label = next_element_1.get_attribute('aria-label')
                            with open('selenium_log.txt', 'a', encoding='utf-8') as f:
                                f.write(f"{second_span_aria_label}\n")
                            
                            # 将符合条件的数据写入文件
                            with open('filtered_aria_label_data.txt', 'a', encoding='utf-8') as f:
                                f.write(f"符合模式的元素组 {i//4}:\n")
                                f.write(f"a标签href: {a_href}\n")
                                f.write(f"a标签text: {a_text}\n")
                                f.write(f"第二个span的aria-label: {second_span_aria_label}\n")
                                f.write("=" * 50 + "\n")
                            
                            # print(f"找到符合模式的元素组 {i//4}: a标签href={a_href}, text={a_text}, 第二个span的aria-label={second_span_aria_label}")
                            if second_span_aria_label and (('天' in second_span_aria_label) or ('小时' in second_span_aria_label) or ('分钟' in second_span_aria_label)):
                                with open('selenium_log.txt', 'a', encoding='utf-8') as f:
                                    f.write('--加入列表--\n')
                                single_new_list.append((href_value,a_text))
                        except Exception as e:
                            with open('selenium_log.txt', 'a', encoding='utf-8') as f:
                                f.write(f"处理符合模式的元素组时出错: {e}\n")
    if len(single_new_list)>0:
            specific_news[hot_text]=single_new_list
    else:
        with open('selenium_log.txt', 'a', encoding='utf-8') as f:
            f.write(f'{hot_text}查看更多模式下没有找到符合循环模式的新闻url\n')
    #再次打开页面，获取查看完整新闻模式的url
    #打开热点列表中的一个
    single_new_list=[]
    driver.get(url)
    time.sleep(1)
    try:
        # 获取新页面中第一个有aria-label属性，且属性值中有'查看完整'的a标签中，取得a标签的href值
        elements_with_aria_label = driver.find_elements(by=By.CSS_SELECTOR, value="[aria-label]")
        # 查找第一个包含'查看完整'的a标签
        for element in elements_with_aria_label:
            aria_label_value = element.get_attribute('aria-label')
            tag_name = element.tag_name
            # 检查是否是a标签且aria-label包含'查看完整'
            if tag_name == 'a' and aria_label_value and '查看完整' in aria_label_value:
                href_value = element.get_attribute('href')
                with open('selenium_log.txt', 'a', encoding='utf-8') as f:
                    f.write(f"找到包含'查看完整'的a标签，href: {href_value}\n")
                break
        with open('selenium_log.txt', 'a', encoding='utf-8') as f:
            f.write(f"查看完整模式下找到的新闻列表入口，href: {href_value}\n")
        if href_value:
            driver.get(href_value)
            time.sleep(1)
        # 获取页面所有class中含有content的a标签（class包含content但不一定完全等于content）
        elements_with_content_class = driver.find_elements(by=By.CSS_SELECTOR, value="a[class*='content']")
        for element in elements_with_content_class:
            href_value = element.get_attribute('href')
            text_content = element.text.strip()
            single_new_list.append((href_value,text_content))
            # class_value = element.get_attribute('class')
            # html_code = element.get_attribute('outerHTML')
            # with open('content_class_data.txt', 'a', encoding='utf-8') as f:
            #     f.write(f"元素 {i}:\n")
        if len(single_new_list)>0:
                if hot_text in specific_news:
                    specific_news[hot_text]=specific_news[hot_text]+single_new_list
                else:
                    specific_news[hot_text]=single_new_list
                with open('selenium_log.txt', 'a', encoding='utf-8') as f:
                    f.write(f'{hot_text}查看完整模式下找到符合循环模式的新闻url：\n{single_new_list}\n')
        else:
            with open('selenium_log.txt', 'a', encoding='utf-8') as f:
                f.write(f'{hot_text}查看更多模式下没有找到符合循环模式的新闻url\n')
    except Exception as e:
        with open('selenium_log.txt', 'a', encoding='utf-8') as f:
            f.write(f"处理查看完整模式时出错: {e}\n")

    
    
    
    
    
    




In [32]:
hot_href_list

['https://www.baidu.com/s?wd=9%E6%9C%883%E6%97%A5%E6%98%AF%E4%B8%80%E4%B8%AA%E5%80%BC%E5%BE%97%E6%B0%B8%E8%BF%9C%E7%BA%AA%E5%BF%B5%E7%9A%84%E6%97%A5%E5%AD%90&sa=fyb_n_homepage&rsv_dl=fyb_n_homepage&from=super&cl=3&tn=baidutop10&fr=top1000&rsv_idx=2&hisfilter=1',
 'https://www.baidu.com/s?wd=%E7%94%B7%E7%94%9F%E5%A4%8D%E8%AF%BB3%E5%B9%B4%E9%AB%98%E8%80%83%E6%9F%A5%E5%88%86%E5%96%9C%E6%9E%81%E8%80%8C%E6%B3%A3&sa=fyb_n_homepage&rsv_dl=fyb_n_homepage&from=super&cl=3&tn=baidutop10&fr=top1000&rsv_idx=2&hisfilter=1',
 'https://www.baidu.com/s?wd=%E5%90%84%E5%9C%B0%E9%AB%98%E8%80%83%E5%88%86%E6%95%B0%E7%BA%BF%E5%87%BA%E7%82%89&sa=fyb_n_homepage&rsv_dl=fyb_n_homepage&from=super&cl=3&tn=baidutop10&fr=top1000&rsv_idx=2&hisfilter=1',
 'https://www.baidu.com/s?wd=%E9%82%B1%E8%B4%BB%E5%8F%AF%E5%8B%89%E5%8A%B1%E8%80%83%E7%94%9F%EF%BC%9A%E6%8F%A1%E7%B4%A7%E4%BA%BA%E7%94%9F%E5%8F%91%E7%90%83%E6%9D%83&sa=fyb_n_homepage&rsv_dl=fyb_n_homepage&from=super&cl=3&tn=baidutop10&fr=top1000&rsv_idx=2&hisfilter=1'

In [11]:
news_text = {}
for key in specific_news.keys():
    print(f"=========处理{key}的新闻======")
    url_list =[]
    info_list =[]
    for href,text in specific_news[key]:
        url_list.append(href)
    for url in url_list:
        try:
            res = selenium_trafilatura_extract(url=url,driver=driver)
            #页面标题、文本、作者、网址、链接、日期、长度
            info_list.append((res['page_title'],res['advanced_text'],res['metadata']['author'],res['metadata']['sitename'],res['url'],res['metadata']['date'],res['statistics']['advanced_text_length']))
        except Exception as e:
            print(f" {e}")
    news_text[key] = info_list

=========处理5男生复读3年高考查分喜极而泣的新闻======
🌐 正在访问: http://www.baidu.com/link?url=OZ3RsX98r2ZxOZNySn1PI5dNfMqERZufsYuN_OnrEiYc7zwqNg02ubCRd2Ge935g30yVih3HchSMmNtVr6Pm1NujxcBSv37o2jTDsKWvFpTrAxXtqO9EhmXiE7Evxn_vpxgNJaO71Wmrbs1wNwlNg_RhLbQRoP1MtnMSkKfNWoEWQ0JQCrW5LRMTqJcIj0ulWvCbCvuBIP1iSUAF-hQvJEHKb4r1Ai3mLHu3yDF-75uSOPxY-QnDEt9k7qv_Z5dc
📄 页面标题: 百度资讯搜索_男生复读3年高考查分喜极而泣
📝 HTML长度: 392757 字符
🔍 使用 Trafilatura 提取内容...
✅ 内容提取成功!
   基础提取: 1086 字符
   高级提取: 1124 字符
   提取标题: 男生复读3年高考查分喜极而泣 姐姐:他为考心仪学校顶着压力复读...
   内容预览: 2小时前男生复读3年高考查分喜极而泣 姐姐:他为考心仪学校顶着压力复读,比去年高了70分 6月24日,河北沧州一网友发布视频称弟弟复读三年,终于如愿以偿了。视频中,男生喜极而泣,妈妈也激动地抱住......
🌐 正在访问: http://www.baidu.com/link?url=OZ3RsX98r2ZxOZNySn1PI5dNfMqERZufsYuN_OnrEiYc7zwqNg02ubCRd2Ge935g30yVih3HchSMmNtVr6Pm1NujxcBSv37o2jTDsKWvFpTrAxXtqO9EhmXiE7Evxn_vpxgNJaO71Wmrbs1wNwlNg_RhLbQRoP1MtnMSkKfNWoEWQ0JQCrW5LRMTqJcIj0ulWvCbCvuBIP1iSUAF-hQvJEHKb4r1Ai3mLHu3yDF-75uSOPxY-QnDEt9k7qv_Z5dc
📄 页面标题: 百度资讯搜索_男生复读3年高考查分喜极而泣
📝 HTML长度: 392655 字符
🔍 使用 Trafilatura 提取内容...
✅ 内容提取成功!
   基础提取: 1086

In [10]:
specific_news.keys()

dict_keys(['5男生复读3年高考查分喜极而泣', '2女生高考611分 体检不合格恐无法入学', '7湖北高二学生参加高考 数学满分', '36部门发布19条举措加强金融支持', '8中方回应特朗普要中国买美国石油', '4一个班3人高考成绩屏蔽 老师们沸腾', '9A股收盘：沪指创年内新高'])

In [31]:
specific_news['2女生高考611分 体检不合格恐无法入学']

[('http://www.baidu.com/link?url=mJlKMk-oYXZGiMH3WN8mCHFHH8-3p9wTFBifJSMSP1cdC-7dbJLIiOth-C2fdPxCRPfeKeJ841iiJxn46y1juBnnu3ZZA9C3WEiN5WMOGbuYZWla6NZgEvn-jUcjEGUQr9zJ7jes4h2ZDjYLGh58LK9Yr_3IMa_9NO-arT06o6TRj7FiDLDerhJTqCfapW8nzRrhjW0N7Hs9rBYo7XB5gq7wHRwcyQJm09raaTlv0f7jjMqdjRrAVyOZRHhJEMZNinBqiS-gYvSS_yWd1E0As_',
  '高考611分女生因患病影响志愿填报,当地残联:已帮忙联系院校'),
 ('http://www.baidu.com/link?url=mJlKMk-oYXZGiMH3WN8mCHFHH8-3p9wTFBifJSMSP1cdC-7dbJLIiOth-C2fdPxCRPfeKeJ841iiJxn46y1juBnnu3ZZA9C3WEiN5WMOGbuYZWla6NZgEvn-jUcjEGUQr9zJ7jes4h2ZDjYLGh58LK9Yr_3IMa_9NO-arT06o6TRj7FiDLDerhJTqCfapW8nzRrhjW0N7Hs9rBYo7XB5gq7wHRwcyQJm09raaTlv0f7jjMqdjRrAVyOZRHhJEMZNinBqiS-gYvSS_yWd1E0As_',
  '高考611分女生体检不合格恐无法入学 多方呼吁助力填报志愿'),
 ('http://www.baidu.com/link?url=mJlKMk-oYXZGiMH3WN8mCHFHH8-3p9wTFBifJSMSP1cdC-7dbJLIiOth-C2fdPxCRPfeKeJ841iiJxn46y1juBnnu3ZZA9C3WEiN5WMOGbuYZWla6NZgEvn-jUcjEGUQr9zJ7jes4h2ZDjYLGh58LK9Yr_3IMa_9NO-arT06o6TRj7FiDLDerhJTqCfapW8nzRrhjW0N7Hs9rBYo7XB5gq7wHRwcyQJm09raaTlv0f7jjMqdjRrAVyOZRHhJEMZNinBqiS-gYv

In [12]:
from openai import OpenAI

def get_topics(news_text):
    topics_list = []
    for key in news_text.keys():
        info_list = news_text[key]
        #拼接文本
        raw_texts = ''
        for i,info in enumerate(info_list):
            raw_texts += f'第{i+1}篇：'+'\n'+info[1] + '\n\n'
        #获取topics
        GET_TOPICS_PROMPT = """
        下面是关于“{key}”的一些新闻报道，根据这些新闻，进行新闻内容总结，总结内容不超过50字，并提取出5个主题关键词。
        返回json数据。
        示例返回形式：{"abstrct":"人社部宣布，自从2025年6月起，个人养老金需要缴纳3%的个人所得税",
        "keywords":"民生，税务，政府，养老金，上调税率"}
        新闻报道是：{raw_texts}。
        你的回答：
        """
        GET_TOPICS_PROMPT = GET_TOPICS_PROMPT.replace('{key}',key)
        GET_TOPICS_PROMPT = GET_TOPICS_PROMPT.replace('{raw_texts}',raw_texts)
        print('GET_TOPICS_PROMPT',GET_TOPICS_PROMPT)
        client = OpenAI(api_key="sk-1ce00a653d2c46238249e685eb3a9c7d", base_url="https://api.deepseek.com")
        response = client.chat.completions.create(
        model="deepseek-chat",
        messages=[
            {"role": "system", "content": "You are a helpful assistant"},
            {"role": "user", "content":GET_TOPICS_PROMPT },
        ],
        stream=False
        )
        system_msg = response.choices[0].message.content
        topics_list.append(system_msg)
    return topics_list
res = get_topics(specific_news)
    

GET_TOPICS_PROMPT 
        下面是关于“5男生复读3年高考查分喜极而泣”的一些新闻报道，根据这些新闻，进行新闻内容总结，总结内容不超过50字，并提取出5个主题关键词。
        返回json数据。
        示例返回形式：{"abstrct":"人社部宣布，自从2025年6月起，个人养老金需要缴纳3%的个人所得税",
        "keywords":"民生，税务，政府，养老金，上调税率"}
        新闻报道是：第1篇：
男生复读3年高考查分喜极而泣 姐姐:他为考心仪学校顶着压力复读...

第2篇：
男生复读3年高考查分喜极而泣:为考心仪学校顶着压力复读

第3篇：
男生复读3年高考查分喜极而泣 家人:一直是他坚强的后盾

第4篇：
男生复读3年高考查分喜极而泣,姐姐:比去年高了70分左右

第5篇：
男生复读三年高考查分喜极而泣,姐姐:他为考心仪学校顶着压力复读...

。
        你的回答：
        
GET_TOPICS_PROMPT 
        下面是关于“2女生高考611分 体检不合格恐无法入学”的一些新闻报道，根据这些新闻，进行新闻内容总结，总结内容不超过50字，并提取出5个主题关键词。
        返回json数据。
        示例返回形式：{"abstrct":"人社部宣布，自从2025年6月起，个人养老金需要缴纳3%的个人所得税",
        "keywords":"民生，税务，政府，养老金，上调税率"}
        新闻报道是：第1篇：
高考611分女生因患病影响志愿填报,当地残联:已帮忙联系院校

第2篇：
高考611分女生体检不合格恐无法入学 多方呼吁助力填报志愿

第3篇：
高考611分女生体检不合格恐无法入学,守护她的求学梦也是守护教育...

第4篇：
高考611分女生体检不合格恐无法入学 多方呼吁助力填报志愿(2)

第5篇：
高考611分女生体检不合格恐无法入学 残联回应

第6篇：
女生高考611分,因患脊髓性肌萎缩症体检不合格,无法正常填报志愿...

第7篇：
...年报考在京招生公安院校公安专业考生参加政治考察、面试、体检...

第8篇：
四川:百问百答③ | 2025年考生关注的100个志愿填报问题

第9篇：
山东公布普通高校

In [24]:
#写入数据库
import psycopg2
from psycopg2 import Error
##获取连接对象
def connect_to_postgres():
    try:
        # 连接到PostgreSQL数据库
        connection = psycopg2.connect(
            user="postgres",
            password="wzdshjw123",
            host="localhost",
            port="5432",
            database="first_pg"
        )
        return connection
    except Error as e:
        print(f"连接数据库时出错: {e}")
        return None

#执行查询
def execute_query(query, params=None):
    try:
        connection = connect_to_postgres()
        if connection is None:
            return
        
        cursor = connection.cursor()
        
        # 执行查询
        if params:
            cursor.execute(query, params)
        else:
            cursor.execute(query)
            
        # 如果是SELECT查询，获取结果
        if query.strip().upper().startswith('SELECT'):
            results = cursor.fetchall()
            # 获取列名
            column_names = [desc[0] for desc in cursor.description] if cursor.description else []
            # print("列名:", column_names)
            # print("查询结果:", results)
            return results
        else:
            # 对于INSERT、UPDATE、DELETE操作，获取受影响的行数
            affected_rows = cursor.rowcount
            print(f"操作影响的行数: {affected_rows}")
            
        # 提交事务
        connection.commit()
        
    except Error as e:
        print(f"执行查询时出错: {e}")
    finally:
        if connection:
            cursor.close()
            connection.close()
            print("数据库连接已关闭")

In [15]:
import json
json_list =[json.loads(json_str.replace('```json\n', '').replace('\n```', '')) for json_str in res]

In [20]:
json_list[0]['keywords']

'复读，高考，压力，家庭支持，喜极而泣'

In [22]:

import json

# 原始字符串
json_str = '```json\n{\n"abstrct":"男生复读3年高考查分喜极而泣，姐姐称他为考心仪学校顶着压力复读，家人是其坚强后盾。",\n"keywords":"复读，高考，压力，家庭支持，喜极而泣"\n}\n```'

# 移除markdown代码块标记
cleaned_str = json_str.replace('```json\n', '').replace('\n```', '')

# 解析JSON
parsed_json = json.loads(cleaned_str)

print("解析后的JSON对象:")
print(parsed_json)
print(f"\n摘要: {parsed_json['abstrct']}")
print(f"关键词: {parsed_json['keywords']}")


解析后的JSON对象:
{'abstrct': '男生复读3年高考查分喜极而泣，姐姐称他为考心仪学校顶着压力复读，家人是其坚强后盾。', 'keywords': '复读，高考，压力，家庭支持，喜极而泣'}

摘要: 男生复读3年高考查分喜极而泣，姐姐称他为考心仪学校顶着压力复读，家人是其坚强后盾。
关键词: 复读，高考，压力，家庭支持，喜极而泣


In [ ]:
   id SERIAL PRIMARY KEY,
    hottopic TEXT,
    page_title TEXT,
    content_text TEXT,
    author TEXT,
    site TEXT,
    url TEXT,
    update_time DATE,
    content_length INTEGER,
	absrtact TEXT,
	keywords TEXT,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP

In [28]:
# 将news_text写入txt文件
with open('news_text.txt', 'w', encoding='utf-8') as f:
    for hottopic, news_list in news_text.items():
        f.write(f"热点话题: {hottopic}\n")
        f.write("="*50 + "\n")
        for i, news_item in enumerate(news_list, 1):
            page_title, content_text, author, site, url, update_time, content_length = news_item
            f.write(f"新闻 {i}:\n")
            f.write(f"标题: {page_title}\n")
            f.write(f"内容: {content_text}\n")
            f.write(f"作者: {author}\n")
            f.write(f"来源: {site}\n")
            f.write(f"链接: {url}\n")
            f.write(f"更新时间: {update_time}\n")
            f.write(f"内容长度: {content_length}\n")
            f.write("-"*30 + "\n")
        f.write("\n" + "="*50 + "\n\n")

print("news_text已成功写入news_text.txt文件")


news_text已成功写入news_text.txt文件


In [26]:
import datetime
# 准备插入数据的SQL语句 (PostgreSQL使用%s占位符)
insert_sql = """
INSERT INTO baidu_news (hottopic, page_title, content_text, author, site, url, update_time, content_length,absrtact,keywords)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s,%s,%s)
"""

# 遍历news_text字典，将数据插入数据库
i = 0
for hottopic, news_list in news_text.items():
    abstract = json_list[i]['abstrct']
    keywords = json_list[i]['keywords']
    for news_item in news_list:
        try:
            # 解包元组数据
            page_title, content_text, author, site, url, update_time, content_length = news_item
            
            # 处理可能的None值
            page_title = page_title if page_title else ''
            content_text = content_text if content_text else ''
            author = author if author else ''
            site = site if site else ''
            url = url if url else ''
            
            # 特殊处理日期字段 - 确保是有效的日期格式或NULL
            if update_time and update_time.strip() and update_time != '':
                # 如果有有效的日期，直接使用
                processed_update_time = update_time
            else:
                # 如果日期为空或无效，使用当前日期
                processed_update_time = datetime.datetime.now().strftime('%Y-%m-%d')
            
            content_length = content_length if content_length else 0
            
            # 执行插入操作
            execute_query(insert_sql, (hottopic, page_title, content_text, author, site, url, processed_update_time, content_length,abstract,keywords))
            print(f"成功插入: {hottopic} - {page_title}")
            
            
        except Exception as e:
            print(f"插入失败 {hottopic}: {e}")
    i+=1

print("数据插入完成！")




操作影响的行数: 1
数据库连接已关闭
成功插入: 5男生复读3年高考查分喜极而泣 - 百度资讯搜索_男生复读3年高考查分喜极而泣
操作影响的行数: 1
数据库连接已关闭
成功插入: 5男生复读3年高考查分喜极而泣 - 百度资讯搜索_男生复读3年高考查分喜极而泣
操作影响的行数: 1
数据库连接已关闭
成功插入: 5男生复读3年高考查分喜极而泣 - 百度资讯搜索_男生复读3年高考查分喜极而泣
操作影响的行数: 1
数据库连接已关闭
成功插入: 5男生复读3年高考查分喜极而泣 - 百度资讯搜索_男生复读3年高考查分喜极而泣
操作影响的行数: 1
数据库连接已关闭
成功插入: 5男生复读3年高考查分喜极而泣 - 百度资讯搜索_男生复读3年高考查分喜极而泣
操作影响的行数: 1
数据库连接已关闭
成功插入: 2女生高考611分 体检不合格恐无法入学 - 百度资讯搜索_女生高考611分 体检不合格恐无法入学
操作影响的行数: 1
数据库连接已关闭
成功插入: 2女生高考611分 体检不合格恐无法入学 - 百度资讯搜索_女生高考611分 体检不合格恐无法入学
操作影响的行数: 1
数据库连接已关闭
成功插入: 2女生高考611分 体检不合格恐无法入学 - 百度资讯搜索_女生高考611分 体检不合格恐无法入学
操作影响的行数: 1
数据库连接已关闭
成功插入: 2女生高考611分 体检不合格恐无法入学 - 百度资讯搜索_女生高考611分 体检不合格恐无法入学
操作影响的行数: 1
数据库连接已关闭
成功插入: 2女生高考611分 体检不合格恐无法入学 - 百度资讯搜索_女生高考611分 体检不合格恐无法入学
操作影响的行数: 1
数据库连接已关闭
成功插入: 2女生高考611分 体检不合格恐无法入学 - 百度资讯搜索_女生高考611分 体检不合格恐无法入学
操作影响的行数: 1
数据库连接已关闭
成功插入: 2女生高考611分 体检不合格恐无法入学 - 百度资讯搜索_女生高考611分 体检不合格恐无法入学
操作影响的行数: 1
数据库连接已关闭
成功插入: 2女生高考611分 体检不合格恐无法入学 - 百度资讯搜索_女生高考611分 体检不合格恐无法入学
操作影响的行数: 1
数据库连接已关闭
成功插入: 2女生高考611分 体检不合格恐无法入学 - 百度资讯搜索_女生高考61

In [ ]:
drop table baidu_news;
CREATE TABLE IF NOT EXISTS baidu_news (
    id SERIAL PRIMARY KEY,
    hottopic TEXT,
    page_title TEXT,
    content_text TEXT,
    author TEXT,
    site TEXT,
    url TEXT,
    update_time DATE,
    content_length INTEGER,
	absrtact TEXT,
	keywords TEXT,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP

In [10]:
test_href = href_value_list[0]
driver.get(test_href)
title = driver.title
print(f"最新消息页面标题: {title}")

最新消息页面标题: 百度资讯搜索_9月3日是一个值得永远纪念的日子


In [15]:
# 获取页面所有包含aria-label属性的元素
elements_with_aria_label = driver.find_elements(by=By.CSS_SELECTOR, value="[aria-label]")

print(f"找到 {len(elements_with_aria_label)} 个包含aria-label属性的元素")
print("=" * 80)
single_new_list=[]
# 遍历每个元素并打印其原始HTML代码
for i, element in enumerate(elements_with_aria_label, 1):
    try:
        # 获取元素的aria-label属性值
        aria_label_value = element.get_attribute('aria-label')
        # 获取元素的原始HTML代码
        html_code = element.get_attribute('outerHTML')
        
        # 将数据写入txt文件
        with open('aria_label_data.txt', 'a', encoding='utf-8') as f:
            f.write(f"元素 {i}:\n")
            f.write(f"aria-label值: {aria_label_value}\n")
            f.write(f"HTML代码: {html_code}\n")
            f.write("-" * 50 + "\n")
    except Exception as e:
        print(f"处理第 {i} 个元素时出错: {e}")
        print("-" * 50)    
    # 检查元素结构，寻找a标签-span标签-span标签-span标签的模式
    if i >= 2:  # 从第三个元素开始检查
        # 检查当前元素及其后3个元素是否形成a-span-span-span模式
        if i + 3 <= len(elements_with_aria_label):  # 确保有足够的后续元素
            current_element = element
            next_element_1 = elements_with_aria_label[i]  # 当前元素
            next_element_2 = elements_with_aria_label[i+1]  # 后一个元素
            next_element_3 = elements_with_aria_label[i+2]  # 后两个元素
            
            # 检查标签类型
            current_tag = current_element.tag_name
            next_tag_1 = next_element_1.tag_name
            next_tag_2 = next_element_2.tag_name
            next_tag_3 = next_element_3.tag_name
            
            # 检查是否符合 a-span-span-span 的模式
            if (current_tag == 'a' and next_tag_1 == 'span' and 
                next_tag_2 == 'span' and next_tag_3 == 'span'):
                
                try:
                    # 获取第一个a标签的href和text
                    a_href = current_element.get_attribute('href')
                    a_text = current_element.text.strip()
                    
                    # 获取第二个span标签的aria-label值
                    second_span_aria_label = next_element_1.get_attribute('aria-label')
                    print(second_span_aria_label)
                    
                    # 将符合条件的数据写入文件
                    with open('filtered_aria_label_data.txt', 'a', encoding='utf-8') as f:
                        f.write(f"符合模式的元素组 {i//4}:\n")
                        f.write(f"a标签href: {a_href}\n")
                        f.write(f"a标签text: {a_text}\n")
                        f.write(f"第二个span的aria-label: {second_span_aria_label}\n")
                        f.write("=" * 50 + "\n")
                    
                    print(f"找到符合模式的元素组 {i//4}: a标签href={a_href}, text={a_text}, 第二个span的aria-label={second_span_aria_label}")
                    if second_span_aria_label and (('天' in second_span_aria_label) or ('小时' in second_span_aria_label) or ('分钟' in second_span_aria_label)):
                        print('--加入列表--')
                        single_new_list.append((href_value,a_text))
                except Exception as e:
                    print(f"处理符合模式的元素组时出错: {e}")
    

print("=" * 80)
print(single_new_list)


找到 43 个包含aria-label属性的元素
发布于：4小时前
找到符合模式的元素组 0: a标签href=https://www.jfdaily.com/wx/detail.do?id=935260, text=主播说联播丨9月3日是一个值得永远纪念的日子, 第二个span的aria-label=发布于：4小时前
--加入列表--
发布于：2024年9月3日
找到符合模式的元素组 1: a标签href=https://history.sohu.com/a/806023229_121106822, text=今天,是一个值得世界人民永远纪念的日子, 第二个span的aria-label=发布于：2024年9月3日
发布于：5天前
找到符合模式的元素组 2: a标签href=https://history.sohu.com/a/906151705_121165488, text=日本明明是8月15日投降,为何抗战胜利纪念日,却是9月3日?, 第二个span的aria-label=发布于：5天前
--加入列表--
发布于：2024年9月3日
找到符合模式的元素组 4: a标签href=https://opinion.dahe.cn/2024/09-03/1811463.html, text=【地评线】河声丨9月3日!我们该怎么纪念?, 第二个span的aria-label=发布于：2024年9月3日
发布于：2024年9月4日
找到符合模式的元素组 5: a标签href=https://www.ntv.cn/content/1/324/991324896.html, text=农视网, 第二个span的aria-label=发布于：2024年9月4日
发布于：2024年9月3日
找到符合模式的元素组 6: a标签href=https://news.china.com/socialgd/10000169/20240903/47149358.html, text=致敬中国人民的伟大胜利 勿忘历史,珍爱和平, 第二个span的aria-label=发布于：2024年9月3日
发布于：2016年9月3日
找到符合模式的元素组 7: a标签href=http://www.xinhuanet.com/politics/2016-09/03/c_129268353.htm, te

In [ ]:
#打开更多消息页面，一般可以打开
for href_value in href_value_list:
    if href_value:
        driver.get(href_value)
    else:
        print("跳过此热点，未找到有效链接")
        continue
    time.sleep(1)
    #获取最新消息页面的标题
    title = driver.title
    print(f"最新消息页面标题: {title}")
    #获取最新消息页面上的所有div中的具体消息url，div可能也找不到，需要处理
    

    result_divs = driver.find_elements(by=By.CSS_SELECTOR, value="div.result-op.c-container.xpath-log.new-pmd")
    print(f"找到 {len(result_divs)} 个符合条件的div元素")
    # print("=" * 80)
    single_new_list=[]
    # 遍历每个div元素，取出其中的url，按照更新时间进行筛选
    for div in result_divs: 
        try:
            # 在当前div中查找a标签
            a_tag = div.find_element(by=By.TAG_NAME, value="a")
            href_value = a_tag.get_attribute('href')
            link_text = a_tag.text.strip()
            #这里会发生报错，找不到这个选择器
            span_element = div.find_element(by=By.CSS_SELECTOR, value="span.c-color-gray2.c-font-normal.c-gap-right-xsmall")
            span_aria_label = span_element.get_attribute('aria-label')
            print(span_aria_label)
            if span_aria_label and (('天' in span_aria_label) or ('小时' in span_aria_label) or ('分钟' in span_aria_label)):
                print('--加入列表--')
                single_new_list.append((href_value,link_text))
    
                
        except Exception as e:
            print(f"  处理第 {1} 个div时出错: {e}")
        
        # print("=" * 80)
    if single_new_list:
        specific_news[hot_text]=single_new_list
news_text = {}
for key in specific_news.keys():
    print(f"=========处理{key}的新闻======")
    url_list =[]
    info_list =[]
    for href,text in specific_news[key]:
        url_list.append(href)
    for url in url_list:
        try:
            res = selenium_trafilatura_extract(url=url,driver=driver)
            #页面标题、文本、作者、网址、链接、日期、长度
            info_list.append((res['page_title'],res['advanced_text'],res['metadata']['author'],res['metadata']['sitename'],res['url'],res['metadata']['date'],res['statistics']['advanced_text_length']))
        except Exception as e:
            print(f" {e}")
    news_text[key] = info_list